# 目錄

## 自我介紹
- 哈嘍大家好我是凱文
- 這是我的YouTube頻道： https://www.youtube.com/@aidiffusion99
- 歡迎訂閱留言！

  
## 本次代碼實現的內容：
1. **如何使用混元模型生成人物一致性視頻 （Huyuan + Lora + TeaCache加速）**
2. **如何使用谷歌雲**
3. 本次視頻地址：

## 其他相關的視頻列表：
1. 《如何註冊和使用谷歌雲， 新用戶免費送$1300美金》
     - 視頻地址：https://youtu.be/GsJWWnPcWFU?si=Il4m122U6i0YQudG
2. 《如何使用TeaCache加速Flux 300%》
     - 視頻地址：https://youtu.be/SjBLo7YN958?si=68DddafnnrVW7sWK
3. 《如何使用使用TeaCache加速混元視頻模型，加速200%》
     - 視頻地址：https://youtu.be/NLukkCU292I?si=EeHVKUxmUdIIfus6

## 你需要使用的工具列表
1. Civitai - 下載模型
    1. 網址：https://civitai.com
    2. 註冊一個帳號，並創建token（視頻裡有演示如何創建），token大概是這樣子的: fyew78c0b23dc193d17307ea28ds798wwf （這個token是假的，只是展示下token長什麼樣子）。你要把它保存起來，不要丟失了。
    3. 如何30秒獲得 CivitAI網站的個人帳號token，請看這個視頻 https://youtu.be/KCzRWGbOB5M?si=ByUfCDSdp8BFtgQt&t=180

# 第一步：啟動系統
- 這一步是安裝ComfyUI和常用插件
- 運行時間大約是1-5分鐘，取決於你的網絡連結情況

In [ ]:
import subprocess
import os
from os import path
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')
from multiprocessing import Process
import sys
import time


###### 把你Civitai的網站token拷貝下下面
##### 這一步必須要設置，否則無法從Civitai下載模型
CIVITAI_TOEKN = "a85ec0b23dc193d17307ea2898cb8f6a"




HF_TOK = 'abc'
BASE = '/home/jupyter'

def download_from_civitai(url, folder, name):
    tok = CIVITAI_TOEKN
    model_url = f"{url}&token={tok}"
    save_path = f"{BASE}/ComfyUI/models/{folder}/{name}"
    get_ipython().system(f'wget -O "{save_path}" "{model_url}"')
    %cd $BASE

def delete_model(folder, name):
    %cd $BASE/ComfyUI/models/$folder
    all_names = os.listdir()
    print("當前所在的路徑是: ", os.getcwd())
    print("當前目錄內的所有文件: ", all_names)

    if name in all_names:
        %rm $name
        print("模型文件已經被成功刪除了✅")
    else:
        print("*** 沒有找到你提供的模型文件❌")
    %cd $BASE
    
def install_plg(git_url):
    %cd $BASE/ComfyUI/custom_nodes
    !git clone $git_url
    print("下載完成✅")
    %cd $BASE

%cd $BASE
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
get_ipython().system('git pull')
#!pip install --force-reinstall torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu124
!pip install -r requirements.txt
!pip install --upgrade gguf
!pip install bitsandbytes>=0.43.0
!mamba install openssh -y
!pip install -U huggingface_hub
!export HF_ENDPOINT=https://hf-mirror.com

%cd $BASE/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
get_ipython().system('git pull')
!pip install -r requirements.txt

%cd $BASE/ComfyUI/custom_nodes
!git clone https://github.com/AIGODLIKE/AIGODLIKE-ComfyUI-Translation.git
!git clone https://github.com/chrisgoringe/cg-use-everywhere.git
# !git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git
# !git clone https://github.com/WASasquatch/was-node-suite-comfyui.git
!git clone https://github.com/rgthree/rgthree-comfy.git
# !git clone https://github.com/city96/ComfyUI-GGUF
!git clone https://github.com/crystian/ComfyUI-Crystools.git
%cd ComfyUI-Crystools
!pip install -r requirements.txt

%cd $BASE/ComfyUI/custom_nodes
!git clone https://github.com/kijai/ComfyUI-HunyuanVideoWrapper.git
%cd ComfyUI-HunyuanVideoWrapper
!pip install -r requirements.txt

# put it here to make it easy to use for people who do not code
# sd1.5 vae file, 0.3gb
# url = "https://civitai.com/api/download/models/311162?type=Model&format=SafeTensor"
# folder = "vae"
# name = "vae-ft-mse-840000-ema-pruned.safetensors"
# download_from_civitai(url, folder, name)

# !git clone https://github.com/LarryJane491/Lora-Training-in-Comfy.git
# !git clone https://github.com/kijai/ComfyUI-KJNodes.git
# !git clone https://github.com/11cafe/comfyui-workspace-manager.git
# %cd /kaggle/working/ComfyUI/custom_nodes
# !git clone https://github.com/kijai/ComfyUI-FluxTrainer.git
# %cd ComfyUI-FluxTrainer
# !pip install -r requirements.txt

clear_output(wait=True)
%cd $BASE/ComfyUI

print(f"Current path: {BASE}/ComfyUI")
print("***** 已經成功安裝ComfyUI和常用插件, 第一步已完成 ✅😄")

# 第二步： 下載模型
- 下載Hunyuan模型和lora模型

In [ ]:
### 兩個clip文件
%cd $BASE/ComfyUI/models/clip
model_url = "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/clip_l.safetensors?download=true"
model_name = "clip_l.safetensors"
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

### 9G
model_url = "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/llava_llama3_fp8_scaled.safetensors?download=true"
model_name = "llava_llama3_fp8_scaled.safetensors"
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

### 一個vae文件
model_url = "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/vae/hunyuan_video_vae_bf16.safetensors?download=true"
model_name = "hunyuan_video_vae_bf16.safetensors"
%cd $BASE/ComfyUI/models/vae
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

### 一個模型文件，顯卡內存夠大，就選擇bf16。如果不夠，就選擇fb8
# bf16模型，26G
# %cd $BASE/ComfyUI/models/unet
# model_url = "https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/diffusion_models/hunyuan_video_t2v_720p_bf16.safetensors?download=true"
# model_name = "hunyuan_video_t2v_720p_bf16.safetensors"
# get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# # fp8小模型文件，13G
%cd $BASE/ComfyUI/models/unet
model_url = "https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_720_cfgdistill_fp8_e4m3fn.safetensors?download=true"
model_name = "hunyuan_video_720_cfgdistill_fp8_e4m3fn.safetensors"
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')


###### 默認下載下面6個lora模型 ######
## 1. Lisa 混元lora模型
url = "https://civitai.com/api/download/models/1274571?type=Model&format=SafeTensor"
folder = "loras"
name = "Lisa.safetensors"
download_from_civitai(url, folder, name)

## 2. Emilia Clarke 權利的遊戲 龍媽
url = "https://civitai.com/api/download/models/1273832?type=Model&format=SafeTensor"
folder = "loras"
name = "LongMa.safetensors"
download_from_civitai(url, folder, name)

## 3. Yori
url = "https://civitai.com/api/download/models/1184914?type=Model&format=SafeTensor"
folder = "loras"
name = "Yori.safetensors"
download_from_civitai(url, folder, name)

## 4. fukada
url = "https://civitai.com/api/download/models/1282806?type=Model&format=SafeTensor"
folder = "loras"
name = "fukada.safetensors"
download_from_civitai(url, folder, name)

## 5. swim
url = "https://civitai.com/api/download/models/1197482?type=Model&format=SafeTensor"
folder = "loras"
name = "swim.safetensors"
download_from_civitai(url, folder, name)

## 6. gantz
url = "https://civitai.com/api/download/models/1252600?type=Model&format=SafeTensor"
folder = "loras"
name = "gantz.safetensors"
download_from_civitai(url, folder, name)

# 第三步：啟動ComfyUI網頁

- 當你看到下面的顯示就表示已經成功啟動了
> Starting server, To see the GUI go to: ttp://127.0.0.1:xxxx

In [ ]:
%cd $BASE/ComfyUI

!touch log.txt
open('log.txt', 'w').close()
clear_output(wait=True)

def run_app():
    cmd = f"python {BASE}/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)

def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()

    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)

p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()